In [1]:
import os
import ast
import tqdm
import pandas as pd

In [2]:
reports_references = pd.read_csv("data/opencti_reports_external_references.csv")

In [3]:
references_dict = {
    x["ID"]: ast.literal_eval(x["Ext. Reference URLs"])
    for _, x in reports_references.iterrows()
}

In [4]:
from docling.document_converter import DocumentConverter
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import requests
import time

converter = DocumentConverter()

def extract_report(source):
    result = converter.convert(source)
    try:
        # Extract image URLs
        response = requests.get(source)
        soup = BeautifulSoup(response.content, 'html.parser')
        image_urls = [img['src'] for img in soup.find_all('img') if 'src' in img.attrs]
    except:
        image_urls = []
    
    image_urls = "\n".join(image_urls)

    if image_urls:
        final_result = f"{result.document.export_to_markdown()}\n\n# Image URLs #\n\n{image_urls}"
    else:
        final_result = result.document.export_to_markdown()

    return final_result

def get_soup_object(url, use_table_counter=True):
    try:
        options = Options()
        #options.add_argument('--headless')  # Run in headless mode (no window)
        options.add_argument('--disable-gpu')
        
        # 1. Initialize Selenium WebDriver (adjust path to your webdriver)
        driver = webdriver.Chrome(options=options)  # Or any other browser driver
        
        # 2. Navigate to the webpage containing the table
        driver.get(url)

        if use_table_counter:

            # # 3. Locate the <select> element that controls the number of items
            # select_element = WebDriverWait(driver, 10).until(
            #     EC.presence_of_element_located((By.ID, "count"))  # Replace with the actual ID
            #     # Or use other locators like By.NAME, By.XPATH, By.CSS_SELECTOR
            # )

            select_element = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.ID, "count"))
            )
            
            item_selector = Select(select_element)

            # 4. Select the desired number of items
            desired_item_count = "100"  # Replace with the value you want to select (e.g., "50", "All")
            item_selector.select_by_value(desired_item_count)  # Or select_by_visible_text() or select_by_index()

            # 5. Wait for the table to load/update after the selection (important!)
            WebDriverWait(driver, 10)
            # .until(
            #     EC.presence_of_element_located((By.XPATH, "//table[@id='your_table_id']/tbody/tr"))  # Adjust XPath to target a row in the table
            # )
            time.sleep(2)  # Add an extra small delay if needed for rendering

        # 6. Get the updated HTML source
        html_source = driver.page_source
        soup = BeautifulSoup(html_source, 'html.parser')
    finally:
        # 8. Close the browser
        driver.quit()
    return soup

def extract_alienvault_report(url, use_table_counter=True):

    soup = get_soup_object(url, use_table_counter)
    
    # Get title
    title = soup.find("h1").get_text()

    # Get metadata
    metadata = soup.find("div", class_="pulse-meta").find('ul').get_text().strip()

    # Get contents
    contents = "\n\n".join(
        [tag.get_text().strip() for tag in soup.find("div", class_="pulse-general-details").find_all("div", class_="row col-md-12 col-flex detail-row ng-star-inserted")]
        )

    # Get table
    table_rows = soup.find("otx-table").find_all("tr")

    table_dict = {}
    for i, tr in enumerate(table_rows):
        if i==0:
            table_columns = []
            for tag in tr.find_all("div", class_="clickable"):
                table_dict.update({tag.get_text().strip():[]})
                table_columns.append(tag.get_text().strip())
        else:
            for j, tag in enumerate(tr.find_all("td", class_="ng-star-inserted")):
                k = table_columns[j]
                table_dict[k].append(tag.get_text().strip())

    if "" in table_dict.keys():
        table_dict.pop("")

    table_str = pd.DataFrame(table_dict).to_string(index=False, header=True, na_rep='--', col_space=2)

    alienvault_report = "# Title #\n{title}\n\n# Metadata #\n{metadata}\n\n# Contents #\n{contents}\n\n# Indicators of Compromise #\n{table}".format(
        title = title,
        metadata = metadata,
        contents = contents,
        table = table_str
    )
    return alienvault_report

c:\Users\AIAS\Documents\cti-model-training\venv-eda\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
import concurrent.futures

for k, v in tqdm.tqdm(references_dict.items()):

    doc = ""

    raw_path = os.path.join("data/reports_formatted/raw", k)
    alienvault_path = os.path.join("data/reports_formatted/alienvault", k)

    paths_exist = 0
    for p in [raw_path, alienvault_path]:
        if os.path.exists(p):
            paths_exist += 1
            continue
        else:
            os.mkdir(p)

    if paths_exist==2:
        continue

    for i, url in enumerate(v):

        if url.startswith("https://otx.alienvault.com/"):
            save_path = f"data/reports_formatted/alienvault/{k}"
            extract = extract_alienvault_report
        else:
            save_path = f"data/reports_formatted/raw/{k}"
            extract = extract_report
        
        with concurrent.futures.ThreadPoolExecutor() as executor:
            future = executor.submit(extract, url)

            try:
                report = future.result(timeout=15)
            except Exception as e:
                report = "REPORT IS NOT EXTRACTED! Reason caused the failure:\n\n{e}".format(
                    e = e
                )
        doc = "External reference URL: {url}\n\nCTI REPORT\n\n{report}\n\n".format(
            url = url,
            report = report
        )

        with open(os.path.join(save_path, f"ref_{i}"), mode="w", encoding="utf-8") as f:
            f.write(doc)